This Jupyter Notebook is meant to outline the basic structure of WaterTAP flowsheet

In [2]:
# Start with importing relevant libraries

from pyomo.environ import (
    ConcreteModel,
    check_optimal_termination,
    assert_optimal_termination,
    units as pyunits,
)
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.core.util.model_diagnostics.infeasible import *
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump

from watertap.core.solvers import get_solver

In [15]:
# Build a concentrate model and flowsheet block
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)


In [16]:
# Add the property package relevant to the unit process to the flowsheet
m.fs.properties = SeawaterParameterBlock()

In [17]:
# Add unit process model to the flowsheet. We will start with a pump model
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

In [ ]:
m.fs.pump.control_volume.properties_in.calculate_state(
    var_args={
        ("flow_vol_phase", "Liq"): 1,
        ("conc_mass_phase_comp", ("Liq", "TDS")): 0.5,
        ("pressure", None): 101325,
        ("temperature", None): 298,
    },
    hold_state=True,
)

# m.fs.feed.properties.calculate_state(
#         var_args={
#             ("flow_vol_phase", "Liq"): flow_in,  # m3/s
#             ("conc_mass_phase_comp", ("Liq", target_ion)): conc_mass_in,  # kg/m3
#             ("pressure", None): 101325,
#             ("temperature", None): 298,
#         },
#         hold_state=True,
#     )

2025-10-16 15:50:05 [INFO] idaes.init.fs.pump.control_volume.properties_in: fs.pump.control_volume.properties_in State Released.
2025-10-16 15:50:05 [WARNING] idaes.watertap.property_models.seawater_prop_pack: While using the calculate_state method on fs.pump.control_volume.properties_in[0.0], variable flow_vol_phase was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calculate_scaling_factor method has not been used or the variable was created on demand after the scaling factors were calculated. It is recommended to touch all relevant variables (i.e. call them or set an initial value) before using the calculate_scaling_factor method.
2025-10-16 15:50:05 [WARNING] idaes.watertap.property_models.seawater_prop_pack: While using the calculate_state method on fs.pump.control_volume.properties_in[0.0], variable conc_mass_phase_comp was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calcula

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 8, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.0357511043548584}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
# Provide unit model inputs. For the pump we need to provide inlet flow, inlet pressure, outlet pressure, temperature and efficiency

# Add pyunits

feed_flow_mass = 1 # kg/s
feed_mass_frac_TDS = 0.035
feed_mass_frac_H2O = 1 - feed_mass_frac_TDS

m.fs.pump.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(
    feed_flow_mass * feed_mass_frac_TDS
)
m.fs.pump.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    feed_flow_mass * feed_mass_frac_H2O
)

feed_pressure_in = 1e5 # Pa
feed_pressure_out = 5e5 # Pa
feed_temperature = 273.15 + 25 # K

m.fs.pump.inlet.pressure[0].fix(feed_pressure_in)
m.fs.pump.inlet.temperature[0].fix(feed_temperature)
m.fs.pump.outlet.pressure[0].fix(feed_pressure_out)

m.fs.pump.efficiency_pump.fix(0.75)

In [20]:
m.fs.pump.inlet.display()

inlet : Size=1
    Key  : Name                 : Value
    None : flow_mass_phase_comp : {(0.0, 'Liq', 'H2O'): 996.8115848891376, (0.0, 'Liq', 'TDS'): 0.4999999836724598}
         :             pressure : {0.0: 101325}
         :          temperature : {0.0: 298}


In [25]:
# Lets check the degrees of freedom of the flowsheet! This should ideally be zero now.
degrees_of_freedom(m)

0

In [ ]:
# Let's initialize the flowsheet
m.fs.pump.initialize()

In [ ]:
# Let's solve the flowsheet now!
solver = get_solver()
results = solver.solve(m, tee=False)
# Ensure that the solver status is ok and optimal termination
assert_optimal_termination(results)

ipopt-watertap: ipopt with user variable scaling and IDAES jacobian constraint scaling
Ipopt 3.13.2: tol=1e-08
constr_viol_tol=1e-08
acceptable_constr_viol_tol=1e-08
bound_relax_factor=0.0
honor_original_bounds=no
nlp_scaling_method=user-scaling


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, 

In [ ]:
def build():
    return m

m.fs.pump.properties_in.calculate_state(
    var_args={
        ("flow_vol_phase", ["Liq"]): m.fs.pump.inlet.flow_vol_phase[0, "Liq"],
        ("conc_mass_phase_comp", ["Liq", "TDS"]): m.fs.pump.inlet.conc_mass_phase_comp[0, "Liq", "TDS"],
        ("pressure"): feed_pressure_in,
        ("temperature"): feed_temperature,
    }
)